# PARTE 1

In [1]:
import json
import pandas as pd
from urllib.parse import urlparse

# 1. Cargar la información del archivo large_even.json
with open("large_even.json", "r") as file:
    data = [json.loads(line) for line in file]

print(f"Total de registros: {len(data)}")  # Debería ser 746909

# 2. Filtrar solo los registros DNS
dns_records = [entry for entry in data if entry.get("event_type") == "dns"]
print(f"Cantidad de registros DNS: {len(dns_records)}")  # Debería ser 21484

# 3. Mostrar dos registros DNS de ejemplo
print(dns_records[:2])

# 4. Normalizar la información
from pandas import json_normalize

df = json_normalize(dns_records)
print(f"Shape del DataFrame: {df.shape}")  # Debería ser (21484, 163)

# 5. Filtrar solo registros de tipo A
df_a = df[df["dns.rrtype"] == "A"]
print(f"Cantidad de registros tipo A: {df_a.shape[0]}")  # Debería ser 2849

# 6. Filtrar dominios únicos
domains = df_a["dns.rrname"].unique()
print(f"Cantidad de dominios únicos: {len(domains)}")  # Debería ser 177

# 7. Función para extraer el TLD
def get_tld(domain):
    parts = domain.split('.')
    if len(parts) > 2:
        return '.'.join(parts[-2:])
    return domain

# Verificar función con ejemplos
examples = ["api.wunderground.com", "safebrowsing.clients.google.com.home"]
for ex in examples:
    print(f"{ex} -> {get_tld(ex)}")

# 8. Crear DataFrame con dominios únicos y extraer TLD
df_domains = pd.DataFrame({"domain": domains})
df_domains["domain_tld"] = df_domains["domain"].apply(get_tld)

# Mantener solo la columna domain_tld
df_domains = df_domains[["domain_tld"]]

print(df_domains.head())

Total de registros: 746909
Cantidad de registros DNS: 15749
[{'timestamp': '2017-07-22T17:33:16.661646-0500', 'flow_id': 1327836194150542, 'pcap_cnt': 22269, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 59680, 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 15529, 'rrname': 'api.wunderground.com', 'rrtype': 'A', 'tx_id': 0}}, {'timestamp': '2017-07-22T17:33:24.990320-0500', 'flow_id': 2022925111925872, 'pcap_cnt': 54352, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 38051, 'dest_ip': '2001:0500:0003:0000:0000:0000:0000:0042', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 58278, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}]
Shape del DataFrame: (15749, 18)
Cantidad de registros tipo A: 2849
Cantidad de dominios únicos: 177
api.wunderground.com -> wunderground.com
safebrowsing.cli

In [2]:
df_domains.head()

,domain_tld
0,wunderground.com
1,dropbox.com
2,aoltw.net
3,com.home
4,mozilla.com


In [3]:
df_domains.shape

(177, 1)

In [4]:
import json
import pandas as pd
from pandas import json_normalize

# Cargar datos JSON línea por línea
def load_json(file_path):
    with open(file_path, 'r') as f:
        return [json.loads(line) for line in f]

# Filtrar eventos DNS
def filter_dns_events(data):
    return [entry for entry in data if entry.get('event_type', '').lower() == 'dns']

# Convertir a DataFrame
def json_to_dataframe(data):
    return json_normalize(data)

# Analizar registros tipo A y dominios únicos
def analyze_dns_data(df):
    dns_type_a = df[df['dns.rrtype'] == 'A']
    unique_domains = dns_type_a['dns.rrname'].nunique()
    return dns_type_a.shape[0], unique_domains

# Función para obtener el TLD
def get_tld(domain):
    parts = domain.strip('.').split('.')
    if len(parts) > 2:
        return '.'.join(parts[-2:])
    return domain

# Ejecutar análisis
file_path = 'large_even.json'
data = load_json(file_path)
dns_events = filter_dns_events(data)
df_dns = json_to_dataframe(dns_events)

total_a_records, unique_domains_count = analyze_dns_data(df_dns)

# Crear DataFrames separados
df_domain = df_dns[['dns.rrname']].dropna().rename(columns={'dns.rrname': 'domain'})
df_domain_tld = df_domain.copy()
df_domain_tld['domain_tld'] = df_domain_tld['domain'].apply(get_tld)

df_domain_tld.head()

,domain,domain_tld
0,api.wunderground.com,wunderground.com
1,stork79.dropbox.com,dropbox.com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,aoltw.net
3,hpca-tier2.office.aol.com.ad.aol.aoltw.net,aoltw.net
4,<root>,<root>


In [5]:
df_domain

,domain
0,api.wunderground.com
1,stork79.dropbox.com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net
3,hpca-tier2.office.aol.com.ad.aol.aoltw.net
4,<root>
...,...
15744,en-us.start3.mozilla.com
15745,<root>
15746,whitecell.localdomain
15747,whitecell.localdomain


In [6]:
print(df.columns)

Index(['timestamp', 'flow_id', 'pcap_cnt', 'event_type', 'vlan', 'src_ip',
       'src_port', 'dest_ip', 'dest_port', 'proto', 'dns.type', 'dns.id',
       'dns.rrname', 'dns.rrtype', 'dns.tx_id', 'dns.rcode', 'dns.ttl',
       'dns.rdata'],
      dtype='object')


In [7]:
print(df['dns.rrtype'].unique() if 'dns.rrtype' in df.columns else "Columna dns.rrtype no encontrada")
print(df['dns.type'].unique() if 'dns.type' in df.columns else "Columna dns.type no encontrada")


['A' nan 'SOA' 'AAAA' 'PTR' 'TXT' 'SRV' 'NS']
['query' 'answer']


# PARTE 2

In [8]:
import os
from dotenv import load_dotenv

# Cargar las variables del archivo .env
load_dotenv()

# Obtener la API KEY
API = os.getenv("GOOGLE_API_KEY")

In [9]:
import google.generativeai as genai
import time

def evaluar_dominio_con_gemini(dominio):
    """
    Determina si un dominio es generado por algoritmo (DGA) o legítimo.
    Retorna 1 si es DGA, 0 si es legítimo. En caso de respuesta ambigua, devuelve None.
    """
    prompt = f"""Clasifica el siguiente dominio como DGA (1) o legítimo (0). 
    Un dominio DGA suele ser una combinación aleatoria de caracteres sin formar palabras reconocibles.
    Devuelve solo un número: 1 para DGA, 0 para legítimo.
    
    Dominio: {dominio}
    """

    modelo = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    respuesta = modelo.generate_content(prompt)
    resultado = respuesta.text.strip()
    
    if "1" in resultado and "0" not in resultado:
        return 1
    elif "0" in resultado and "1" not in resultado:
        return 0
    else:
        print(f"Respuesta ambigua para {dominio}: {resultado}")
        return None



In [10]:

def clasificar_dominio_con_reintentos(dominio, intentos_max=3, pausa=10):
    """
    Clasifica un dominio con reintentos y manejo de errores.
    Retorna 1 si es DGA, 0 si es legítimo, "Ambiguo" si no se puede determinar y "Error" en caso de fallo.
    """
    prompt = f"""
    Clasifica el siguiente dominio como DGA (1) o legítimo (0).  
    Responde únicamente con 0 o 1.  
    Dominio: {dominio}
    """
    
    for intento in range(intentos_max):
        try:
            respuesta = modelo_gemini.generate_content(prompt)
            resultado = respuesta.text.strip()
            
            if resultado == "1":
                return 1
            elif resultado == "0":
                return 0
            else:
                return "Ambiguo"
        except Exception as e:
            print(f"Error en {dominio}: {e} | Intento {intento + 1}/{intentos_max}")
            time.sleep(pausa)
    
    return "Error"

In [11]:
# Inicializar modelo de Gemini
modelo_gemini = genai.GenerativeModel("models/gemini-1.5-flash")

In [12]:
# Lista para almacenar los resultados
detecciones = []

# Clasificación de los dominios
total_dominios = len(df_domains)
for idx, dominio in enumerate(df_domains["domain_tld"]):
    resultado = clasificar_dominio_con_reintentos(dominio)
    detecciones.append({"domain_tld": dominio, "dga_score": resultado})
    
    if isinstance(resultado, int):  
        time.sleep(5)  # Pausa para evitar sobrecarga en el servicio


In [13]:

# Crear DataFrame desde la variable 'resultados' y guardarlo
df_clasificacion = pd.DataFrame(detecciones)
df_clasificacion.to_csv("resultados.csv", index=False)

# Unir con el DataFrame de dominios
df_unidos = df_domains.merge(df_clasificacion, on="domain_tld", how="left")

# Filtrar dominios legítimos y eliminar duplicados
df_legitimos = df_clasificacion[df_clasificacion["dga_score"] == 1]
df_legitimos_unicos = df_legitimos.drop_duplicates(subset=["domain_tld"])

# Cantidad de dominios únicos legítimos
print(f"Total de dominios únicos legítimos: {len(df_legitimos_unicos)}")

# Primeros 10 registros
df_legitimos_unicos.head(10)


Total de dominios únicos legítimos: 7


,domain_tld,dga_score
18,22.110phpmyadmin,1
27,ntkrnlpa.info,1
82,malwarecity.com,1
135,21.1201,1
160,vtlfccmfxlkgifuf.com,1
167,ejfodfmfxlkgifuf.xyz,1
169,22.201:,1


# PARTE 3

In [15]:
import requests

# Cargar la lista de dominios más populares
df_top = pd.read_csv("top-1m.csv", names=["rank", "domain"])
df_top["tld"] = df_top["domain"].str.split(".").str[-1].str.lower()
top_tlds = set(df_top["tld"].unique())

# Función para verificar si un TLD está en la lista
def es_tld_popular(tld):
    return tld in top_tlds

# Filtrar dominios sospechosos
df_dga = df_clasificacion[df_clasificacion["dga_score"] == 1]
df_dga_unicos = df_dga.drop_duplicates(subset=["domain_tld"])
df_dga_unicos["tld"] = df_dga_unicos["domain_tld"].str.split(".").str[-1].str.lower()
df_dga_unicos["es_popular"] = df_dga_unicos["tld"].apply(es_tld_popular)

# Filtrar dominios cuyo TLD es popular
df_filtrados = df_dga_unicos[df_dga_unicos["es_popular"]]

# Función para obtener fecha de creación de un TLD desde RDAP
def obtener_fecha_creacion(tld):
    url = f"https://rdap.org/domain/{tld}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            for evento in response.json().get("events", []):
                if evento.get("eventAction") == "registration":
                    return evento.get("eventDate")
    except:
        pass
    return "Desconocido"

# Obtener fechas de creación
df_filtrados["fecha_creacion"] = df_filtrados["tld"].apply(obtener_fecha_creacion)

# Mostrar resultados
print(df_filtrados[["domain_tld", "tld", "fecha_creacion"]])


               domain_tld   tld fecha_creacion
27          ntkrnlpa.info  info    Desconocido
82        malwarecity.com   com    Desconocido
160  vtlfccmfxlkgifuf.com   com    Desconocido
167  ejfodfmfxlkgifuf.xyz   xyz    Desconocido


/tmp/ipykernel_18061/4192306463.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrados["fecha_creacion"] = df_filtrados["tld"].apply(obtener_fecha_creacion)


## Resultados y Confirmación de Dominios Sospechosos:

    Filtrado de TLDs en la Lista de Dominios Populares: Se utilizó la lista de dominios más populares (top-1m.csv) para determinar si los TLDs de los dominios sospechosos se encuentran en esa lista. Se filtraron los dominios que tienen un TLD presente en esta lista.

    Eliminación de Duplicados: Después de filtrar los dominios por TLD, se eliminaron los duplicados.

    Obtención de Fechas de Creación de los TLDs: Para confirmar la fecha de creación de los TLDs, se utilizó la API de RDAP. En los casos en que no se pudo obtener la fecha de creación, la función devolvió "Desconocido". Las fechas de creación de los TLDs se obtuvieron y se agregaron a los resultados finales.


        ejfodfmfxlkgifuf.xyz: Tiene un patrón aleatorio y puede ser confirmado como un dominio DGA.

    Dominios Confirmados: Los dominios que cumplen con las características de dominios sospechosos pero cuyo TLD está en la lista de dominios más populares, y que no han podido confirmar su fecha de creación, se muestran con la etiqueta "Desconocido".

## Dominios finales con fecha de creación desconocida:

    ntkrnlpa.info - TLD: info

    malwarecity.com - TLD: com

    vtlfccmfxlkgifuf.com - TLD: com

    ejfodfmfxlkgifuf.xyz - TLD: xyz

## Para concluir:

    Confirmados como sospechosos: Los dominios que presentan patrones de secuencias aleatorias de caracteres, como el dominio ejfodfmfxlkgifuf.xyz, pueden ser clasificados como DGA.

    Confirmación de fechas de creación: Algunos TLDs no tienen la fecha de creación disponible, lo que hace que su origen sea más incierto y, por lo tanto, se clasifiquen como "Desconocido".

Este análisis permite refinar los dominios que se consideran como posibles amenazas y que podrían necesitar más verificación.